In [ ]:
HOURS = 1
MINUTES = 0

In [ ]:
from surianalytics.connectors import RESTSciriusConnector

In [ ]:
from surianalytics.widgets.config import update_connector_timeframe

In [ ]:
CONNECTOR = RESTSciriusConnector()

In [ ]:
CONNECTOR.page_size = 100

In [ ]:
update_connector_timeframe(c=CONNECTOR, dump_path="../widgets/config.json")

In [ ]:
STORAGE = "29-uniq-novel-sigs.pkl"

In [ ]:
from IPython.display import JSON

In [ ]:
DATA = CONNECTOR.get_eve_fields_graph(qfilter="event_type: alert", 
                                      col_src="flow.src_ip", 
                                      col_dest="alert.signature",
                                      size_src=1000,
                                      size_dest=1000)

In [ ]:
import os

In [ ]:
import pickle

In [ ]:
if os.path.exists(STORAGE):
    with open(STORAGE, "rb") as handle:
        print("loading pickle")
        SEEN = pickle.load(handle)
else:
    SEEN = dict()

In [ ]:
from datetime import datetime, timedelta

In [ ]:
for edge in DATA["graph"]["edges"]:
    t = tuple(edge["edge"])
    if t not in SEEN:
        print("adding edge", edge["edge"])
        SEEN[t] = datetime.now()

In [ ]:
with open(STORAGE, "wb") as handle:
    pickle.dump(SEEN, handle)

In [ ]:
DELTA = timedelta(hours=HOURS, minutes=MINUTES)

In [ ]:
SOURCES = []
SIGS = []

In [ ]:
for item, seen in SEEN.items():
    if seen > datetime.now() - DELTA:
        SOURCES.append(item[0])
        SIGS.append(item[1])

In [ ]:
import pandas as pd

In [ ]:
DF = pd.DataFrame()

In [ ]:
DF["source"] = SOURCES
DF["signature"] = SIGS

In [ ]:
import ipywidgets as widgets

In [ ]:
def show(limit: int, sort: str):
    pd.set_option('display.max_rows', limit)
    pd.set_option('display.min_rows', limit)
    pd.set_option('display.max_colwidth', None)
    return DF.sort_values(by=[sort])

In [ ]:
widgets.interact(show, 
                 limit=widgets.IntSlider(min=10, max=1000),
                 sort=widgets.Dropdown(options=list(DF.columns.values)))